# 🧪 Test ClinicalBERT / CamemBERT-bio

Ce notebook teste les modèles de text encoding médical.

**Modèles disponibles :**
- `ClinicalBERT` : Anglais (medicalai/ClinicalBERT)
- `CamemBERT-bio` : Français (almanach/camembert-bio-base) ⭐ RECOMMANDÉ

## 1️⃣ Installation des dépendances

In [1]:
# Installer si nécessaire
!pip install transformers torch -q

## 2️⃣ Imports

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")

c:\Users\HP\Desktop\Master2\llm\Triage_urgence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ PyTorch version: 2.10.0+cpu
✅ CUDA disponible: False


## 3️⃣ Test CamemBERT-bio (Français) ⭐

In [3]:
# Charger le modèle français
print("🔧 Chargement CamemBERT-bio...")

model_name = "almanach/camembert-bio-base"
tokenizer_fr = AutoTokenizer.from_pretrained(model_name)
model_fr = AutoModel.from_pretrained(model_name)

# Mettre en mode évaluation
model_fr.eval()

print("✅ Modèle chargé !")

🔧 Chargement CamemBERT-bio...


c:\Users\HP\Desktop\Master2\llm\Triage_urgence\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--almanach--camembert-bio-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 197/197 [00:00<00:00, 826.58it/s, Materializing param=enc

✅ Modèle chargé !


In [4]:
# Tester avec des symptômes en français
symptoms_fr = [
    "douleur thoracique intense",
    "essoufflement au repos",
    "sueurs froides"
]

# Joindre en un texte
text_fr = ", ".join(symptoms_fr)
print(f"📝 Texte d'entrée : {text_fr}")

# Tokenize
inputs = tokenizer_fr(text_fr, return_tensors="pt", max_length=512, truncation=True)
print(f"\n🔢 Tokens : {inputs['input_ids'].shape}")

# Encoder
with torch.no_grad():
    outputs = model_fr(**inputs)

# Extraire embeddings (token [CLS])
embeddings = outputs.last_hidden_state[:, 0, :].numpy()

print(f"\n✅ Embeddings extraits !")
print(f"   Shape : {embeddings.shape}")
print(f"   Type : {embeddings.dtype}")
print(f"   Min : {embeddings.min():.3f}")
print(f"   Max : {embeddings.max():.3f}")
print(f"   Mean : {embeddings.mean():.3f}")
print(f"\n   Premiers 10 valeurs : {embeddings[0][:10]}")

📝 Texte d'entrée : douleur thoracique intense, essoufflement au repos, sueurs froides

🔢 Tokens : torch.Size([1, 16])

✅ Embeddings extraits !
   Shape : (1, 768)
   Type : float32
   Min : -1.855
   Max : 0.524
   Mean : 0.011

   Premiers 10 valeurs : [ 0.07329346  0.19525796  0.07638985 -0.02134542 -0.12308737  0.1294392
  0.02200944  0.14559159  0.11541703 -0.00263739]


## 6️⃣ Créer une fonction réutilisable

In [5]:
def encode_symptoms(symptoms: list, model_type="camembert-bio"):
    """
    Encode une liste de symptômes en embeddings.
    
    Args:
        symptoms: Liste de symptômes
        model_type: "camembert-bio" ou "clinical-bert"
    
    Returns:
        np.ndarray: Embeddings (768,)
    """
    # Joindre symptômes
    text = ", ".join(symptoms)
    
    # Choisir modèle
    if model_type == "camembert-bio":
        tokenizer = tokenizer_fr
        model = model_fr
    else:
        tokenizer = tokenizer_en
        model = model_en
    
    # Encoder
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    
    return embeddings[0]

In [6]:
# Test de la fonction
test_symptoms = ["fièvre élevée", "toux sèche", "fatigue"]

emb = encode_symptoms(test_symptoms)

print(f"✅ Fonction testée !")
print(f"   Input : {test_symptoms}")
print(f"   Output shape : {emb.shape}")
print(f"   Exemple : {emb[:5]}")

✅ Fonction testée !
   Input : ['fièvre élevée', 'toux sèche', 'fatigue']
   Output shape : (768,)
   Exemple : [ 0.05768203  0.19409488  0.11901671 -0.06593066 -0.05924556]
